In [1]:
from langchain.chat_models import ChatOpenAI
from langchain.callbacks import get_openai_callback
from langchain.schema import HumanMessage

llm = ChatOpenAI(model="gpt-4-1106-preview")

text = """
### Instruction

楽曲『約束はいらない』でデビューした、声優は誰？
Respond with reference to the following Context.
The response should output only the words that are the answer.

### Context

(0.21018645, '約束はいらない', '「約束はいらない」(やくそくはいらない)は、坂本真綾のデビューシングル。') (0.24241784, '約束はいらない', '坂本真綾の歌手デビュー作品。当時坂本はまだ無名の声優であったが、同曲がテーマソングとなったアニメ『天空のエスカフローネ』とともに知名度を上げることとなる。後に「指輪」が同アニメの劇場版映画の主題歌とな') (0.2441598, '坂本真綾', '本格的な歌手活動は、1996年にテレビアニメ『天空のエスカフローネ』へ出演したことがきっかけで始めており、同作のオープニングテーマソングである「約束はいらない」(岩里祐穂作詞、菅野よう子作曲)をシング') (0.26043588, 'Suara', '大阪府立豊中高等学校を経て大阪外国語大学でインドネシア語を専攻中にバンド・ユニットを組んで音楽活動を始めた。普段はお笑い番組が大好きなこともあってよく喋るほうだが、東京の仕事で標準語の喋りをする時は、') (0.2690969, '早見沙織', '声優デビュー時より、数多くの主題歌やキャラクターソングを担当し、バラードからポップス、ヒットソングのカバー曲や英語の楽曲など、様々な曲を歌いこなす。2009年には吉田仁美とのユニット「blue dro')

### Answer: 
"""
with get_openai_callback() as cb:
    messages = [HumanMessage(content=text)]
    res = llm.invoke(messages)  # type: ignore

In [2]:
res

AIMessage(content='坂本真綾')

In [3]:
res.content

'坂本真綾'

In [4]:
cb

Tokens Used: 649
	Prompt Tokens: 642
	Completion Tokens: 7
Successful Requests: 1
Total Cost (USD): $0.0066300000000000005

In [5]:
RESPONSE_MESSAGE = "Answer"
RESPONSE_PROMPT = f"### {RESPONSE_MESSAGE}: \n"


def build_prompt(
    user_message: str,
    inputs: str | None = "",
    separator: str = "\n\n### ",
    response_message: str = RESPONSE_MESSAGE,
) -> str:
    prompt = ""
    roles = ["Instruction", response_message]
    messages = [": \n" + user_message, ": \n"]

    if inputs:
        roles = ["Instruction", "Context", response_message]
        messages = [": \n" + user_message, ": \n" + inputs, ": \n"]

    for role, message in zip(roles, messages):
        prompt += separator + role + message
    return prompt

In [6]:
import pandas as pd
import datasets

ds = datasets.load_dataset("hotchpotch/jaqket_v1_qa_wikija_context")  # type: ignore
valid_ds = ds["validation"]  # type: ignore
valid_df = valid_ds.data.to_pandas()  # type: ignore
# context は list なので、 "\n" で結合する
valid_df["context"] = valid_df["context"].apply(lambda x: "\n".join(x) + "\n")
valid_df.head(1)

/home/yu1/miniconda3/envs/llm-sc/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


,qid,question,answer,context,answers,competition,timestamp,section,number,original_question,original_answer,original_additional_info
0,QA20QBIK-0912,手術によって天才となったチャーリイ・ゴードンの日記という形式を通して語られる、ダニエル・キイ...,アルジャーノンに花束を,ダニエル・キイス 「アルジャーノンに花束を」は中編もそれを発展させた長編も、知的障害者のチャ...,[アルジャーノンに花束を],第1回AI王,2020/01/27,開発データ問題 (dev1),912,手術によって天才となったチャーリイ・ゴードンの日記という形式を通して語られる、ダニエル・キイ...,アルジャーノンに花束を,


In [7]:
def qa_prompt(question, context, build_prompt_fn=build_prompt):
    prompt = build_prompt_fn(question, context)
    return prompt

In [8]:
# valid_df での正解率を測る
from tqdm import tqdm

# valid_df = valid_df.head(10)

valid_df = valid_df.reset_index(drop=True)

HINT_MESSAGE = " Respond with reference to the following Context.The response should output only the words that are the answer."

with get_openai_callback() as cb:
    for i in tqdm(range(len(valid_df))):
        prompt = qa_prompt(
            valid_df["question"][i] + HINT_MESSAGE, valid_df["context"][i]
        )
        if i == 0:
            print(prompt)
        answer = valid_df["answer"]
        messages = [HumanMessage(content=prompt)]
        res = llm.invoke(messages)  # type: ignore
        pred = str(res.content)
        valid_df.loc[i, "pred"] = pred.strip()

  0%|          | 0/980 [00:00<?, ?it/s]



### Instruction: 
手術によって天才となったチャーリイ・ゴードンの日記という形式を通して語られる、ダニエル・キイスの小説は何? Respond with reference to the following Context.The response should output only the words that are the answer.

### Context: 
ダニエル・キイス 「アルジャーノンに花束を」は中編もそれを発展させた長編も、知的障害者のチャーリイが実験的手術によって天才となり、短期間でその効果が消えてしまった経過を報告書形式で書いたものである。中編が1959年4月号のF&SF誌に掲載され、1966年に長編化された。この小説は何度か他のメディアに採用されており、1968年の映画『まごころを君に』(出演: クリフ・ロバートソン、クレア・ブルーム)、2002年の日本のテレビドラマ『アルジャーノンに花束を』(出演: ユースケ・サンタマリア、菅野美穂)、などがある。クリフ・ロバートソンはこの作品でアカデミー主演男優賞を受賞している。また、キイスは中編版で1959年にヒューゴー賞、長編版で1966年にネビュラ賞を受賞した。また2015年にも日本のテレビドラマ『アルジャーノンに花束を』(出演:山下智久、窪田正孝など)で新版長編も放送された。
ダニエル・キイス ニューヨーク州ブルックリン区生まれ。17歳のとき U.S. Maritime Service に就職し船員となった。その後しばらくして、働きながら心理学や文学に興味を抱くようになりブルックリンカレッジで心理学の学士号を取得し、一時期ファッション写真のスタジオで働き、その後ニューヨークの高校で国語教師を務めつつ、定時制で英米文学を学び、週末には小説を書いていた。最終的に英米文学の修士号を得ている。1950年代初め、パルプ雑誌『マーヴェル・サイエンス・ストーリーズ』の編集者となった。この雑誌の出版者であるマーティン・グッドマン(英語版)はアメリカン・コミックスも手がけており、マーベル・コミックの前身となったタイムリー・コミック(1940年代)やアトラス・コミック(1950年代)を出版していた。
アルジャーノンに花束を 『アルジャーノンに花束を』(アルジャーノンにはなたばを、Flow

100%|██████████| 980/980 [19:12<00:00,  1.18s/it]  


In [9]:
cb

Tokens Used: 961818
	Prompt Tokens: 955755
	Completion Tokens: 6063
Successful Requests: 980
Total Cost (USD): $9.73944000000002

In [10]:
# 完全一致の正解率を表示
valid_df["is_correct"] = valid_df["answer"] == valid_df["pred"]
valid_df["is_correct"].mean()

0.8785714285714286

In [11]:
# 間違ったものだけを表示
incorrect_df = valid_df[["question", "answer", "pred", "is_correct"]][
    valid_df["is_correct"] == False
]
incorrect_df

,question,answer,pred,is_correct
4,テレビ番組の検証映像でもよく使用される、映した物体の温度を色分けして表示する装置を何というで...,サーモグラフィー,熱線映像装置,False
8,ラテン語では「アーリア・ヤクタ・エスト」という、カエサルがルビコン川を渡る際に言ったとされる...,賽は投げられた,Alea iacta est,False
10,本場鹿児島では「つけあげ」と呼ばれる揚げ物は何でしょう?,薩摩揚げ,さつま揚げ,False
18,端午の節句には鯉のぼりと一緒にあげる、ひらひらとたなびく筒状の布を何というでしょう?,吹流し,鯉の吹き流し,False
27,「エステサロン」などというときのエステとは、何という言葉の略でしょう?,エステティック,エステティックサロン,False
...,...,...,...,...
951,日本語では「工場制手工業」という、機械制大工業が出現する以前に行われていた最初の資本主義的生...,マニュファクチュア,manufacture,False
958,弦を地面と水平に張っている、コンサートなどに使用される大型のピアノを何というでしょう?,グランドピアノ,コンサートグランドピアノ,False
964,旅客機の座席で、ファーストクラスとエコノミークラスの中間にあたるものを一般に何クラスというで...,ビジネスクラス,プレミアム・エコノミー,False
972,新潟県に本社を置く製菓会社で、主力商品である柿の種やハッピーターンで知られるのは何?,亀田製菓,浪花屋製菓,False


In [12]:
# 部分一致の正解率を表示
valid_df["is_correct"] = valid_df.apply(lambda x: x["answer"] in x["pred"], axis=1)
valid_df["is_correct"].mean()

0.9173469387755102

In [13]:
valid_df[["question", "answer", "pred", "is_correct"]].head(100)
# 間違ったものだけを表示
valid_df[["question", "answer", "pred", "is_correct"]][valid_df["is_correct"] == False]

,question,answer,pred,is_correct
4,テレビ番組の検証映像でもよく使用される、映した物体の温度を色分けして表示する装置を何というで...,サーモグラフィー,熱線映像装置,False
8,ラテン語では「アーリア・ヤクタ・エスト」という、カエサルがルビコン川を渡る際に言ったとされる...,賽は投げられた,Alea iacta est,False
10,本場鹿児島では「つけあげ」と呼ばれる揚げ物は何でしょう?,薩摩揚げ,さつま揚げ,False
18,端午の節句には鯉のぼりと一緒にあげる、ひらひらとたなびく筒状の布を何というでしょう?,吹流し,鯉の吹き流し,False
28,鎌倉幕府や室町幕府が荘園を管理支配するために設置した役職で、道理の通じない相手の例えとして「...,地頭,荘官,False
...,...,...,...,...
947,摂氏温度ではマイナス273.15°Cに相当する、熱力学で考えられる最低の温度のことを何という?,絶対零度,Absolute zero,False
950,株式や債券など、金融商品から派生して生まれた「金融派生商品」のことを英語で何という?,デリバティブ,financial derivative products,False
951,日本語では「工場制手工業」という、機械制大工業が出現する以前に行われていた最初の資本主義的生...,マニュファクチュア,manufacture,False
964,旅客機の座席で、ファーストクラスとエコノミークラスの中間にあたるものを一般に何クラスというで...,ビジネスクラス,プレミアム・エコノミー,False


In [14]:
print(
    valid_df[["question", "answer", "pred", "is_correct"]][
        valid_df["is_correct"] == False
    ].to_csv()
)

,question,answer,pred,is_correct
4,テレビ番組の検証映像でもよく使用される、映した物体の温度を色分けして表示する装置を何というでしょう?,サーモグラフィー,熱線映像装置,False
8,ラテン語では「アーリア・ヤクタ・エスト」という、カエサルがルビコン川を渡る際に言ったとされる有名な言葉は何でしょう?,賽は投げられた,Alea iacta est,False
10,本場鹿児島では「つけあげ」と呼ばれる揚げ物は何でしょう?,薩摩揚げ,さつま揚げ,False
18,端午の節句には鯉のぼりと一緒にあげる、ひらひらとたなびく筒状の布を何というでしょう?,吹流し,鯉の吹き流し,False
28,鎌倉幕府や室町幕府が荘園を管理支配するために設置した役職で、道理の通じない相手の例えとして「泣く子とコレには勝てぬ」ともいわれるのは何でしょう?,地頭,荘官,False
29,人間の太ももの裏側にある、大腿四頭筋と対になる大腿二頭筋・半膜様筋・半腱様筋などを総称して、「もも肉のひも」という意味の英語で何という?,ハムストリング,Hamstrings,False
35,クラブなどで人々を踊らせるための電子音楽のことを「EDM」といいますが、これは何という言葉の略でしょう?,エレクトロニック・ダンス・ミュージック,Electronic Dance Music,False
72,兵庫県の「岩津」、埼玉県の「深谷」、群馬県の「下仁田」といえばどんな野菜のブランドでしょう?,ネギ,ねぎ,False
74,アイヌの伝承では柳の葉が人間の世界に流れ出て魚になったという、北海道南部に分布するキュウリウオ科の魚は何?,シシャモ,キュウリウオ,False
83,ティーンエイジャー向けに書かれた小説のことで、英語で「少年向け」という意味があるジャンルを何小説という?,ジュブナイル,ヤングアダルト小説,False
86,マニュアル自動車についている3つのペダルとは、アクセルペダル、ブレーキペダルと何でしょう?,クラッチペダル,クラッチ,False
95,スポーツなどで、声ではなく視線を送ることで合図の代わりにすることを、英語で何というでしょう?,アイコンタクト,eye contact signals,False
99,アフリカの国・シエラレオネの国旗と同じ色の並びの看板を使っ